[![GitHub](https://img.shields.io/badge/Github-hibana2077-blue?style=plastic-square&logo=github)](https://github.com/hibana2077)
[![Colab](https://img.shields.io/badge/Colab-Open%20in%20Colab-blue?style=plastic-square&logo=googlecolab)](https://colab.research.google.com/github/hibana2077/hibana2077/blob/master/train/train.ipynb)

In [ ]:
# !wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
# !tar -xzvf ta-lib-0.4.0-src.tar.gz
# %cd ta-lib
# !./configure --prefix=/usr
# !make
# !make install
# !pip install Ta-Lib

In [30]:
#using sklearn random forest classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from talib import abstract
from ccxt import binance,bybit
import pandas as pd
import numpy as np
import os

In [2]:
os.remove("../data/BTC_USDT_15m.csv")

In [2]:
binance = binance()
symbol = 'BTC/USDT'
timeframe = '15m'
file_name = f"../data/{symbol.replace('/', '_')}_{timeframe}.csv"
start = binance.parse8601('2019-10-01T00:00:00Z')
end = binance.parse8601('2022-12-30T00:00:00Z')
cnt_time = start
data = []
while cnt_time < end:
    ohlcv = binance.fetch_ohlcv(symbol, timeframe, cnt_time)
    data += ohlcv
    cnt_time = ohlcv[-1][0] + 900000 # 1h in ms    
df = pd.DataFrame(data, columns=['time', 'open', 'high', 'low', 'close', 'volume'])
df['time'] = pd.to_datetime(df['time'], unit='ms')
df.to_csv(file_name, index=False)

In [3]:
df

,time,open,high,low,close,volume
0,2019-10-01 00:00:00,8289.97,8310.08,8206.57,8243.39,1016.103644
1,2019-10-01 00:15:00,8242.94,8252.53,8210.01,8220.39,391.576024
2,2019-10-01 00:30:00,8220.78,8222.13,8188.00,8213.23,285.706881
3,2019-10-01 00:45:00,8213.23,8218.34,8195.36,8199.66,176.788254
4,2019-10-01 01:00:00,8199.21,8222.28,8173.05,8211.85,420.860704
...,...,...,...,...,...,...
113817,2022-12-31 07:30:00,16544.76,16561.19,16543.49,16560.64,1262.953190
113818,2022-12-31 07:45:00,16560.64,16568.12,16554.33,16565.09,1261.966080
113819,2022-12-31 08:00:00,16564.81,16567.32,16558.00,16558.93,1127.094570
113820,2022-12-31 08:15:00,16559.18,16562.61,16552.79,16558.35,1181.999050


In [4]:
df['MFI'] = abstract.MFI(df, timeperiod=14)
df['RSI'] = abstract.RSI(df, timeperiod=14)
df['ADX'] = abstract.ADX(df, timeperiod=14)
df['CCI'] = abstract.CCI(df, timeperiod=14)
df['ATR'] = abstract.ATR(df, timeperiod=14)
df['OBV'] = abstract.OBV(df, timeperiod=14)
df['EMA'] = abstract.EMA(df, timeperiod=14)
df['WILLR'] = abstract.WILLR(df, timeperiod=14)
df['AD'] = abstract.AD(df, timeperiod=14)
df['ADOSC'] = abstract.ADOSC(df, timeperiod=14)
df['ADXR'] = abstract.ADXR(df, timeperiod=14)
df['APO'] = abstract.APO(df, timeperiod=14)
df['AROONOSC'] = abstract.AROONOSC(df, timeperiod=14)
df['BOP'] = abstract.BOP(df, timeperiod=14)
df['CCI'] = abstract.CCI(df, timeperiod=14)
df['CMO'] = abstract.CMO(df, timeperiod=14)
df['DX'] = abstract.DX(df, timeperiod=14)
df['MOM'] = abstract.MOM(df, timeperiod=14)
df['PPO'] = abstract.PPO(df, timeperiod=14)
df['ROC'] = abstract.ROC(df, timeperiod=14)
df['ROCP'] = abstract.ROCP(df, timeperiod=14)
df['ROCR'] = abstract.ROCR(df, timeperiod=14)
df['ROCR100'] = abstract.ROCR100(df, timeperiod=14)
df['RSI'] = abstract.RSI(df, timeperiod=14)
df['TRIX'] = abstract.TRIX(df, timeperiod=14)
df['ULTOSC'] = abstract.ULTOSC(df, timeperiod=14)
df['WILLR'] = abstract.WILLR(df, timeperiod=14)
df['WMA'] = abstract.WMA(df, timeperiod=14)
df['HT_TRENDLINE'] = abstract.HT_TRENDLINE(df, timeperiod=14)
df['TRANGE'] = abstract.TRANGE(df, timeperiod=14)
df['CLOSE_percent'] = df['close'].pct_change()

In [5]:
df['UP'] = df['CLOSE_percent'].apply(lambda x: 1 if x > 0 else 0)
df['DOWN'] = df['CLOSE_percent'].apply(lambda x: 1 if x < 0 else 0)
df['UP'] = df['UP'].shift(-1) #shift UP DOWN 一個單位，因為我們要預測的是下一個時間點的漲跌
df['DOWN'] = df['DOWN'].shift(-1)

df = df.dropna()

In [6]:
#正規化
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[['MFI','RSI','ADX','CCI','ATR','OBV','EMA','WILLR','AD','ADOSC','ADXR','APO','AROONOSC','BOP','CCI','CMO','DX','MOM','PPO','ROC','ROCP','ROCR','ROCR100','RSI','TRIX','ULTOSC','WILLR','WMA','HT_TRENDLINE','TRANGE']] = scaler.fit_transform(df[['MFI','RSI','ADX','CCI','ATR','OBV','EMA','WILLR','AD','ADOSC','ADXR','APO','AROONOSC','BOP','CCI','CMO','DX','MOM','PPO','ROC','ROCP','ROCR','ROCR100','RSI','TRIX','ULTOSC','WILLR','WMA','HT_TRENDLINE','TRANGE']])
df[['open','high','low','close','volume']] = scaler.fit_transform(df[['open','high','low','close','volume']])

C:\Users\李軒豪\AppData\Roaming\Python\Python310\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
C:\Users\李軒豪\AppData\Roaming\Python\Python310\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [7]:
X = df.drop(['UP','DOWN','time','CLOSE_percent'], axis=1)
#y -> UP and DOWN
y = df['UP'] #UP

In [8]:
X

,open,high,low,close,volume,MFI,RSI,ADX,CCI,ATR,...,PPO,ROC,ROCP,ROCR,ROCR100,TRIX,ULTOSC,WMA,HT_TRENDLINE,TRANGE
63,0.068547,0.063779,0.070296,0.069147,0.009006,0.545123,0.573676,0.110529,0.663657,0.017247,...,0.665069,0.525270,0.525270,0.525270,0.525270,0.652433,0.483833,0.060348,0.056874,0.005730
64,0.069061,0.063711,0.070232,0.068538,0.005545,0.583632,0.478211,0.097099,0.543233,0.017270,...,0.663999,0.521721,0.521721,0.521721,0.521721,0.652979,0.423186,0.060335,0.056875,0.005697
65,0.068445,0.063783,0.070170,0.069236,0.006517,0.645869,0.569353,0.087159,0.630719,0.017612,...,0.664184,0.530577,0.530577,0.530577,0.530577,0.654181,0.571219,0.060415,0.056885,0.006910
66,0.069142,0.065076,0.070752,0.070208,0.028755,0.666238,0.661885,0.109944,0.828155,0.019634,...,0.666379,0.536803,0.536803,0.536803,0.536803,0.656909,0.610421,0.060617,0.056919,0.013368
67,0.070112,0.065230,0.071492,0.070131,0.014693,0.677433,0.650392,0.133758,0.750029,0.020111,...,0.668514,0.536073,0.536073,0.536073,0.536073,0.660414,0.595304,0.060796,0.056964,0.008062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113816,0.195221,0.190050,0.197228,0.195304,0.018847,0.470188,0.422300,0.150463,0.419275,0.003587,...,0.660592,0.519136,0.519136,0.519136,0.519136,0.655120,0.472882,0.188627,0.185979,0.000826
113817,0.195219,0.190250,0.197245,0.195544,0.031283,0.550820,0.531244,0.136136,0.638452,0.003737,...,0.661073,0.520933,0.520933,0.520933,0.520933,0.655288,0.614744,0.188653,0.185968,0.002481
113818,0.195464,0.190357,0.197413,0.195613,0.031259,0.610873,0.557587,0.127902,0.715838,0.003733,...,0.661454,0.521604,0.521604,0.521604,0.521604,0.655607,0.592934,0.188687,0.185961,0.001933
113819,0.195529,0.190345,0.197470,0.195518,0.027918,0.594242,0.515002,0.120257,0.651462,0.003563,...,0.661597,0.520972,0.520972,0.520972,0.520972,0.655960,0.561749,0.188704,0.185961,0.001307


In [9]:
y.value_counts()

1.0    57230
0.0    56528
Name: UP, dtype: int64

In [10]:
X = X.values
y = y.values

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [12]:
X_train

array([[0.07616831, 0.0710053 , 0.07748153, ..., 0.0676685 , 0.06432281,
        0.00616553],
       [0.18529146, 0.18100395, 0.18674293, ..., 0.1789788 , 0.17562625,
        0.01379884],
       [0.23691367, 0.2320555 , 0.23876938, ..., 0.23101133, 0.22754397,
        0.00539589],
       ...,
       [0.04843619, 0.04305331, 0.04948957, ..., 0.04031012, 0.03686429,
        0.00628749],
       [0.65859559, 0.65750067, 0.66158932, ..., 0.65500844, 0.65230799,
        0.03286739],
       [0.27439417, 0.27277663, 0.27627633, ..., 0.27078841, 0.26937049,
        0.03489873]])

In [13]:
#random forest classifier with n_estimators=10 (default)
model = RandomForestClassifier(
    n_estimators=512)



In [14]:
#train the model using the training sets y_pred=clf.predict(X_test)
model.fit(X_train,y_train)

KeyboardInterrupt: 

In [16]:
pred = model.predict(X_test)

print(accuracy_score(y_test, pred))

0.5306818581425607


In [17]:
pred

array([0., 0., 1., ..., 0., 0., 0.])

In [18]:
X_train[0]

array([0.41947785, 0.41656752, 0.41641046, 0.42092485, 0.04483384,
       0.4149636 , 0.45493847, 0.26179493, 0.3978513 , 0.27413919,
       0.94453874, 0.42048809, 0.36173353, 0.67494628, 0.52553879,
       0.38693563, 0.67816449, 0.75      , 0.59111965, 0.45493847,
       0.31580945, 0.53873752, 0.76601692, 0.53218773, 0.53218773,
       0.53218773, 0.53218773, 0.6786045 , 0.53903802, 0.42316755,
       0.41746466, 0.0693026 ])

In [19]:
test_ans = model.predict(X_train)
print(accuracy_score(y_train, test_ans))

1.0


In [20]:
#confusion matrix
confusion_matrix(y_test, pred)


array([[7404, 6695],
       [6651, 7687]], dtype=int64)

In [21]:
#classification report  
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

         0.0       0.53      0.53      0.53     14099
         1.0       0.53      0.54      0.54     14338

    accuracy                           0.53     28437
   macro avg       0.53      0.53      0.53     28437
weighted avg       0.53      0.53      0.53     28437



In [22]:
#ROC curve
fpr, tpr, thresholds = roc_curve(y_test, pred)
auc = roc_auc_score(y_test, pred)
print('AUC: %.2f' % auc)

AUC: 0.53


In [ ]:
#save model
import pickle
filename = 'model.sav'
pickle.dump(model, open(filename, 'wb'))
